In [1]:
import os
import keras
import keras_hub

In [2]:
os.environ["KERAS_BACKEND"] = "torch"

In [3]:
class Config:
    token_limit = 300
    input_len = 100
    lora_rank = 4
    train_epoch = 20
    learning_rate = 1e-4
    model = "gemma2_instruct_2b_en"

In [4]:
causal_lm = keras_hub.models.CausalLM.from_preset(
    Config.model,
)
causal_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

In [5]:
def generate_text(question):
    template = "<start_of_turn>Q:\n{question}<end_of_turn>\n<start_of_turn>M:"
    message = template.format(question=question)
    output = causal_lm.generate(message, max_length=Config.token_limit)
    print("\nGemma output:")
    print(output)

generate_text("What is self?")


Gemma output:
<start_of_turn>Q:
What is self?<end_of_turn>
<start_of_turn>M:

The concept of "self" is a complex and multifaceted one, with no single, universally accepted definition. It's a question that philosophers, psychologists, and theologians have grappled with for centuries. 

Here are some different perspectives on what "self" might be:

**Philosophical Perspectives:**

* **The Substance Theory:** This view sees the self as a fundamental, unchanging substance that persists through time. It's the core of your being, the "you" that remains constant despite changes in your experiences and thoughts.
* **The Functionalist Theory:** This view emphasizes the role of the self in how we interact with the world. It's not about a fixed substance, but rather about the functions and processes that make up our conscious experience.
* **The Social Constructionist Theory:** This view argues that the self is not an inherent, pre-existing entity, but rather a product of social interactions and

# Dataset Creation

In [6]:
f = open("/kaggle/input/advait-vedant-conversation/data/1.txt", "r")
data = f.readlines()

In [7]:
def reformed_text(text):
    recent_turn = text[:2]
    discourse = f"<start_of_turn>{recent_turn}"
    
    i = 0
    str_len = len(text)
    while i < str_len:
        if text[i:i+2] in ['M:', 'Q:'] and text[i:i+2] != recent_turn:
            discourse += f"<end_of_turn><start_of_turn>"
            recent_turn = text[i:i+2]
        elif text[i:i+2] in ['M:', 'Q:']:
            i += 2
            continue
        discourse += text[i]
        i += 1
    discourse += "<end_of_turn>"
    return discourse

In [8]:
tokenizer = causal_lm.preprocessor.tokenizer


train = []
for x in data:
    item = reformed_text(x)
    length = len(tokenizer(item))
    # skip data if the token length is longer than our limit
    if length < Config.token_limit:
        train.append(item)
        if(len(train) >= Config.input_len):
            break

print(len(train))
print(train[0])

12
<start_of_turn>M: Between the banks of pain and pleasure the river o f life flows. It is only when the mind refuses to flow with life, and gets s tuck at the banks, that it becomes a problem. By flowing with life I mean acceptance - - letting come what comes and go what goes. Desire not, fear not, observe the act ual, as and when it happens, for you are not what happens, you are to whom it happen s. Ultimately even the observer you are not. You are the ultimate potentiality of w hich the all-embracing consciousness is the manifestation and expression.  I am real for I am always now , in the present, and what is with me now shares in my reality. (…) A thing focussed in the no w is with me, for I am ever present; it is my own reality that I impart to the present event.  Things and thoughts have been changing all the time . But the feeling that what is now is real has never changed, even in drea m.  Desire is the memory of pleasure and fear is the me mory of pain. Both make the mind re

In [9]:
data[0]

"M: The sense of being, of 'I am' is the first to emer ge. Ask yourself whence it comes, or just watch it quietly. When the mind stay s in the 'I am' without moving, you enter a state which cannot be verbalised but ca n be experienced. All you need to do is try and try again. After all the sense ‘I am’ is always with you, only you have attached all kinds of things to it -- body, feeling s, thoughts, ideas, possessions etc. All these self-identifications are misleading. Beca use of them you take yourself to be what you are not. M: All you can say is: ‘I am not this, I am not that’ . You cannot meaningfully say ‘this is what I am’. It just makes no sense. What you can point out as 'this' or 'that' cannot be yourself. Surely, you can not be 'something' else. You are nothing perceivable, or imaginable. Yet, without yo u there can be neither perception nor imagination. You observe the heart feeling, the mind thinking, the body acting; the very act of perceiving shows that you are not w hat 

# LoRa Finetuning

In [10]:
#causal_lm.backbone.enable_lora(rank=Config.lora_rank)
causal_lm.summary()

# Limit the input sequence length (to control memory usage).
causal_lm.preprocessor.sequence_length = Config.token_limit

# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=Config.learning_rate,
    weight_decay=0.01,
)

## Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

causal_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

In [11]:
class CustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        model_name = f"/kaggle/working/{lora_name}_{lora_rank}_epoch{epoch+1}.lora.h5"
        causal_lm.backbone.save_lora_weights(model_name)

        # Evaluate
        text_gen("How to be free?")


history = causal_lm.fit(
    train, 
    epochs=Config.train_epoch, 
    batch_size=1, 
    callbacks=[CustomCallback()]
)

import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.show()

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 589824000 elements. This may consume a large amount of memory.
  warnings.warn(


ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/pyt